In [2]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import psycopg2
import configparser
import datetime
import nbimporter
from Functions import *
from Daily_etl import *
from SQL import *
from Connection_to_pgAdmin4 import *

In [3]:
#installation 
#by default you might not have yahoo finance/ library, or psycopg2, or  nbimporternbimporter so you may have to install one
'''pip install yfinance'''
'''pip install nbimporter'''
'''pip install psycopg2'''


'pip install psycopg2'

In [4]:
#schema variables 
#defining these as they will be used for development of sql queries
#source = 'Source'
#staging='Staging'
#dimensions='Dimension'
#facts='Facts

In [5]:
#This is a sample test and will be used for testing only. 
#It utilises yfinance library. It takes "Ticker" symbol then returns the ticker information. 
#Fetch data for a single stock symbol
#This is used to get financial information on stock market from yahoo finance using ticker as test 

stock = yf.Ticker("AAPL")
hist = stock.history(period="25y")  # Fetches one month of historical data

#print(hist)
#If you want to see the result just uncomment the print statement.
#This was a test that was run to ensure that the code is able to work on one ticker


In [15]:
'''
In order to get the data into postgreSQL database there are steps that i am following 

1 In order to download information on a company I will need their ticker name. 

For example company like APPLE their ticker symbol would be AAPL. Their ticker name will be used to feth data based on them. The same applies to the rest of the companies on LSEG. 

The code developed below will use ticker symbol to fetch data per company. Because there is yfinance library available that will help with downloading historical data , 
I will utilise that by using ticker symbol per company, Therefor it is important to use ticker symbol to extract a companies historical data.

There is a file downloaded from https://www.eoddata.com/stockquote/LSE/FEUI.html and an account is required to obtain it.
This file holds information on companies and their tickers.
This is technically easier as there will be less time spent trying to obtain ticker name from stock market.


**************************
Need this to go into log book
'''



'\nIn order to get the data into postgreSQL database there are steps that i am following \n\n1 In order to download information on a company I will need their ticker name. \n\nFor example company like APPLE their ticker symbol would be AAPL. Their ticker name will be used to feth data based on them. The same applies to the rest of the companies on LSEG. \n\nThe code developed below will use ticker symbol to fetch data per company. Because there is yfinance library available that will help with downloading historical data , \nI will utilise that by using ticker symbol per company, Therefor it is important to use ticker symbol to extract a companies historical data.\n\nThere is a file downloaded from https://www.eoddata.com/stockquote/LSE/FEUI.html and an account is required to obtain it.\nThis file holds information on companies and their tickers.\nThis is technically easier as there will be less time spent trying to obtain ticker name from stock market.\n\n\n**************************\

In [10]:


#Moving data from file to the database
#I have now downloaded a file from https://www.eoddata.com/stockquote/LSE/FEUI.html 
#The file contains information on tickers and symbols. 
#I will now pushed information from the file into database.




In [21]:
#Insert data into the table from text file so this can used at a later stage.
#This is being stored in a table so that it can be kept for records. 
#This will also aid in data transformation

#this code will be commented out so incase if run by mistake it does not cause duplications, or rewrites the same table and you may lose data.
#You will need to remove the ******* and uncomment the code. 
#SQL REFERENCE 
##Reference cell


'''
#*******


#path to the text file 
file_path = "lse.txt"

#SQL query to insert data into the table
insert_query = """
INSERT INTO source.ticker_and_name (symbol, Description)
VALUES (%s, %s);
"""

#connection to the database and creates a cursor
connection = connect_to_database()
cursor = connection.cursor()

try:
    # Opens the file for reading
    with open(file_path, "r") as file:
        # Reads each line in the file
        for line in file:
            # Splits each line into columns based on a delimiter (e.g., tab)
            parts = line.strip().split('\t')  # Assumes tab as the delimiter

            # Check if the line contains the expected number of columns
            if len(parts) == 2:  # Assuming 2 columns (Symbol and Description)
                symbol, description = parts

                # Executes the INSERT query for each row
                cursor.execute(insert_query, (symbol, description))

    # Commits the changes to the database
    connection.commit()
    print("Data inserted successfully.")

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")
    if connection:
        connection.rollback()

finally:
    # Closes the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

#*******
'''


#Now that ticker is saved one test will be run below to see what data it holds and get the sape of it.

'\n#*******\n\n\n#path to the text file \nfile_path = "lse.txt"\n\n#SQL query to insert data into the table\ninsert_query = """\nINSERT INTO source.ticker_and_name (symbol, Description)\nVALUES (%s, %s);\n"""\n\n#connection to the database and creates a cursor\nconnection = connect_to_database()\ncursor = connection.cursor()\n\ntry:\n    # Opens the file for reading\n    with open(file_path, "r") as file:\n        # Reads each line in the file\n        for line in file:\n            # Splits each line into columns based on a delimiter (e.g., tab)\n            parts = line.strip().split(\'\t\')  # Assumes tab as the delimiter\n\n            # Check if the line contains the expected number of columns\n            if len(parts) == 2:  # Assuming 2 columns (Symbol and Description)\n                symbol, description = parts\n\n                # Executes the INSERT query for each row\n                cursor.execute(insert_query, (symbol, description))\n\n    # Commits the changes to the da

In [ ]:

#Now that I have ticker and desription of the companies, i will test the sybmols by giving it to yfinance library to fetch infotmation on in.
#Below is an example of the ticker AAPL


In [6]:
#The code below just gets the shape of the metadata on company


# Create a Ticker object for the symbol you want to get metadata for
symbol = "AAPL"  # change it for sampling with the symbol you're want in
ticker = yf.Ticker(symbol)

# Access the metadata attributes
metadata = ticker.info



#***************** 
#more can be printed by removing the comment from the print statament below.
#print (metadata)
print("Metadata for", symbol)
print("Name:", metadata["shortName"])
print("Industry:", metadata["industry"])
print("Sector:", metadata["sector"])
print("Country:", metadata["country"])
print("Market Cap:", metadata["marketCap"])


Metadata for AAPL
Name: Apple Inc.
Industry: Consumer Electronics
Sector: Technology
Country: United States
Market Cap: 2603349704704


In [ ]:
#***********

#Now from the previous test I can get information on tickers I now need to create a table that will hold the information on finance of the companies.
#Below SQL creats a table in the source schema


In [ ]:
#***********

#Now using the information provided by the yfinance library, I will use ticker.history method to call each ticker, and store their information on table that was created above. 
#Below is a function that will be used to fetch data from the yfinance library, because there are so manu columns that I do not need, I will limit the number of columns to fetch too.


In [18]:
#Taking each symbole from the lse.txt file that was downloaded earlier, 
#looping through 1st 100 in the list at a time and fetching records from yahoo finance yfinance library from 2005
#path to file containing symbols
#Read and process the file
 
'''
It will first read the file
and stores the sybmols inside an empty array. 

ensuring the that there are no nulls, or integar values. 

using the stored symbols get companies information since 2005 till today. 

Once records are fetched Then inserting into the table created above

In order to mange hitting the api limits for yfinance library I had to complete this task in batch of a hundred.

'''


In [22]:
#*********************
#WARNING IF YOU RUN THIS CODE IT WILL START POPULATING THE DATABASE THAT WILL RUN FOR FEW DAYS ENSURE YOU HAVE THE RIGHT SPACE AND ENVIRONMENT  
#AND ENSURE THIS CODE REMAIN COMMENTED.
#Functions used 
#SQL REFERENCE A
##Reference cell A
#*********************




'''
# Path to file containing symbols
file_path = "lse.txt"
symbols = []

# Flag to start adding symbols after finding "02NG.L"
start_adding = False

with open(file_path, "r") as file:
    next(file)  # Skip the header
    for line in file:
        symbol = line.split()[0].strip()
        # Skip invalid symbols
        if symbol in ('null', '0.0', ''):
            continue
        if symbol == "02NG.L":
            start_adding = True
        if start_adding:
            symbols.append(symbol)

#SQL INSERT statement here
insert_query = """
    INSERT INTO source.historical_stock_data (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Symbol)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Process symbols in batches of 100
batch_size = 100
for i in range(0, len(symbols), batch_size):
    batch_symbols = symbols[i:i + batch_size]
    all_historical_data = pd.DataFrame()

    for symbol in batch_symbols:
        historical_data = fetch_historical_data(symbol)
        if not historical_data.empty:
            all_historical_data = pd.concat([all_historical_data, historical_data])

    if not all_historical_data.empty:
        records_to_insert = all_historical_data.to_records(index=False).tolist()

        record_count = 0
        for record in records_to_insert:
            # Handle NaN values and convert them to None
            modified_record = tuple([None if pd.isna(value) else value for value in record])

            execute_sql_query(insert_query, modified_record)
            record_count += 1

        print(f"Batch {i//batch_size + 1}: Inserted {record_count} records.")
    else:
        print(f"Batch {i//batch_size + 1}: No data to insert.")
        
'''

'\n# Path to file containing symbols\nfile_path = "lse.txt"\nsymbols = []\n\n# Flag to start adding symbols after finding "02NG.L"\nstart_adding = False\n\nwith open(file_path, "r") as file:\n    next(file)  # Skip the header\n    for line in file:\n        symbol = line.split()[0].strip()\n        # Skip invalid symbols\n        if symbol in (\'null\', \'0.0\', \'\'):\n            continue\n        if symbol == "02NG.L":\n            start_adding = True\n        if start_adding:\n            symbols.append(symbol)\n\n#SQL INSERT statement here\ninsert_query = """\n    INSERT INTO source.historical_stock_data (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Symbol)\n    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)\n"""\n\n# Process symbols in batches of 100\nbatch_size = 100\nfor i in range(0, len(symbols), batch_size):\n    batch_symbols = symbols[i:i + batch_size]\n    all_historical_data = pd.DataFrame()\n\n    for symbol in batch_symbols:\n        historical_data = fe

In [ ]:
'''

The data is now added to the table. 
The next step is to get company information.
This will be achieved using yfinance library method yf.info

There is an example below.

'''

In [7]:



# Using symbols gets information on company eg ticker for Apple, 
symbol = "AAPL"  # Change with the symbol you're want in
ticker = yf.Ticker(symbol)

# Access the metadata attributes
metadata = ticker.info


#just uncomment the print statement to see information on company.
#print (metadata)



In [12]:


#The next step is to get information on companies, as tested above, yf.info method will be used to pull information from yfinance. 


In [ ]:
#*****************
#This code uses both function above and loops through symbols in bathc of 100 and fetches their information 
#Then once information is fetched it is then inserted into the the respective table
#This code will be commented out. To avoid running by error and rewriting the entire table or causing duplication.
#SQL REFERENCE B
#Reference cell B
#*****************
'''

with open("lse.txt", "r", encoding='utf-8') as file:
    next(file)  # Skip the header line
    symbols = [line.split('\t')[0].strip() for line in file]

batch_size = 100
error_symbols = []

for i in range(0, len(symbols), batch_size):
    batch_symbols = symbols[i:i + batch_size]
    for symbol in batch_symbols:
        info_df = fetch_info_data(symbol)
        if info_df is not None and not info_df.empty:
            for _, row in info_df.iterrows():
                insert_customer_data(row)
        else:
            error_symbols.append(symbol)  # Append symbol to error list if data fetch fails

    print(f"Processed batch {i // batch_size + 1}/{len(symbols) // batch_size + 1}")


print("All data processed and inserted into source.s_customer")
if error_symbols:
    with open('s_customer_error_symbols.csv', 'w') as f:
        for symbol in error_symbols:
            f.write(f"{symbol}\n")
    print(f"Symbols with errors written to 's_customer_error_symbols.csv'")

'''

In [ ]:
'''
The above code has run and populated the table needed to be populated. 

The next step is to generate a data dimensions. 
Date dimension is important in order to get STAR SCHEMA the right way for data process, therefor the below code will generate dates and related information, 
such as fisical year, weekends, holidays, quarters.

and a furthur function to insert the data into date table

The table to store the infromation has been already generated.

'''

In [1]:
#*****************
#This code uses the function created generate_date_dimension, and the result of that function is then pushed into the database.
#This code does not need to be run regularley so will comment it out. 
#SQL REFERENCE B
#Reference cell C
#*****************

'''

def insert_date_dimension_row(row):
    query = """
    INSERT INTO source.s_date 
    (Date, Year, Quarter, Month, Week, Day, DayOfWeek, IsWeekend, IsHoliday, FiscalYear, FiscalQuarter, FiscalMonth) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # Formatting the Date to string if it's in datetime format
    date_str = row['Date'].strftime('%Y-%m-%d') if pd.notnull(row['Date']) else None

    params = (
        date_str, row['Year'], row['Quarter'], row['Month'], row['Week'], row['Day'], 
        row['DayOfWeek'], row['IsWeekend'], row['IsHoliday'], row['FiscalYear'], 
        row['FiscalQuarter'], row['FiscalMonth']
    )

    # Execute the SQL query
    execute_sql_query(query, params)

# Generate the date dimension DataFrame
start_date = datetime.date(2005, 1, 1)
end_date = datetime.date(2050, 12, 31)
date_dimension = generate_date_dimension(start_date, end_date)

# Iterate through each row of  DataFrame and insert it
for index, row in date_dimension.iterrows():
    insert_date_dimension_row(row)

'''

'\n\ndef insert_date_dimension_row(row):\n    query = """\n    INSERT INTO source.s_date \n    (Date, Year, Quarter, Month, Week, Day, DayOfWeek, IsWeekend, IsHoliday, FiscalYear, FiscalQuarter, FiscalMonth) \n    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)\n    """\n    # Formatting the Date to string if it\'s in datetime format\n    date_str = row[\'Date\'].strftime(\'%Y-%m-%d\') if pd.notnull(row[\'Date\']) else None\n\n    params = (\n        date_str, row[\'Year\'], row[\'Quarter\'], row[\'Month\'], row[\'Week\'], row[\'Day\'], \n        row[\'DayOfWeek\'], row[\'IsWeekend\'], row[\'IsHoliday\'], row[\'FiscalYear\'], \n        row[\'FiscalQuarter\'], row[\'FiscalMonth\']\n    )\n\n    # Execute the SQL query\n    execute_sql_query(query, params)\n\n# Generate the date dimension DataFrame\nstart_date = datetime.date(2005, 1, 1)\nend_date = datetime.date(2050, 12, 31)\ndate_dimension = generate_date_dimension(start_date, end_date)\n\n# Iterate through each row of  DataFr

In [12]:
stock = yf.Ticker('0JHR.L')
info = stock.history('max')
#print(info)

In [ ]:
'''

Collectiong stock information of symbols that show no stock listing data.

'''

In [14]:
#*********************
#AND ENSURE THIS CODE REMAIN COMMENTED.
#this code is populating the missing data for some companies
#SQL REFERENCE A
#Reference cell G
#*********************

'''
symbols = [
"AAPL",
"HKLB.L",
"IEBB.L",
"LTHP.L",
"ICOV.L",
"IESP.L",
"MCON.L",
"EURO.L",
"FEUD.L",
"GAID.L",
"IEX5.L",
"PUIG.L",
"TFGS.L"
]  # 

for symbol in symbols:
    data = fetch_missing_symbol_data(symbol)
    if not data.empty:
        records_to_insert = [
            (
                row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Volume'],
                row['Dividends'], row['Stock Splits'], row['Symbol']
            )
            for index, row in data.iterrows()
        ]
        
        insert_query = """
            INSERT INTO source.historical_stock_data (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Symbol)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        
        # Execute SQL query for each record
        for record in records_to_insert:
            execute_sql_query(insert_query, record)

'''


In [ ]:
'''
The gaps of missing data has now been completed
There is stock information available now source table. 

'''

In [ ]:
'''

Now since the code part is complete, next thing is to fill the stock data  based on reading table from facts "facts.dates_of_stock_info" that will
compare the last date to todays date and if it is not the same it will fetch the data and append to source and insert it into historical stock data table.

'''